# deck tuner

stuff to make decks better

In [1]:
import sys, os

sys.path.insert(0, os.path.realpath('../'))

In [2]:
import logging

import matplotlib.pyplot as plt
import numpy as np

from IPython.display import Image, display
from ipywidgets import IntSlider, interact
from scipy.stats import hypergeom

from mtg.cards import all_land_card_names
from mtg.extract.tappedout import TappedoutDeck, get_all_categories
from mtg.utils import init_logging

In [3]:
%matplotlib inline

In [4]:
try:
    import seaborn as sns
    sns.set()
except:
    print("default styling")

## logging

In [5]:
LOGGER = logging.getLogger('deck tuner')
LOGGER.setLevel(logging.DEBUG)

In [6]:
init_logging()

## land tuning via hypergeometric calculation

In [7]:
# deck_id = '12-07-19-jeskai-edh'
# deck_id = '06-06-18-esper-blink'
# deck_id = 'sprite-draw'
# deck_id = 'doubling-season-edh'
# deck_id = '23-03-17-gobrins'
# deck_id = 'havoc-festival-edh'
# deck_id = '19-02-17-AGL-breya-edh'
# deck_id = '13-02-16-mizzix-of-the-izmagnus-edh'
deck_id = 'zadaaaaaahhhhhhh-copy'

In [8]:
deck = TappedoutDeck(deck_id=deck_id,
                     ignore_lands=False,
                     with_tags=True)

2019-08-22 17:55:26,630 DEBUG    [mtg.extract.tappedout.get_categories:271] loading categories for deck zadaaaaaahhhhhhh-copy


In [9]:
deck.df.loc[:, 'is_land'] = deck.df.name.isin(all_land_card_names())

main_board = deck.df[deck.df.board == 'main']
main_board.head()

,board,qty,name,printing,foil,alter,signed,condition,languange,tag_list,is_land
0,main,1,Accelerate,TOR,NaN,NaN,NaN,NaN,NaN,NaN,False
1,main,1,Balduvian Rage,CSP,NaN,NaN,NaN,NaN,NaN,NaN,False
2,main,1,Battle Hymn,AVR,NaN,NaN,NaN,NaN,NaN,NaN,False
3,main,1,Beetleback Chief,EMA,NaN,NaN,NaN,NaN,NaN,NaN,False
5,main,1,Boiling Blood,WTH,NaN,NaN,NaN,NaN,NaN,NaN,False


In [10]:
num_cards = main_board.qty.sum()
num_lands = main_board[main_board.is_land].qty.sum()
num_non_lands = num_cards - num_lands
num_drawable = num_cards - 1  # commander

num_cards, num_drawable, num_lands, num_non_lands

(100, 99, 36, 64)

as an example, what is the probability that we will have 3 or more lands in our opening hand of 7 cards with this current deck makeup?

In [11]:
hypergeom.sf(k=3 - 1,
             M=num_drawable,
             n=num_lands,
             N=7)

0.5010769598326307

so what we want to get to now is an interactive where we control deck stats by adding or subtracting lands from the current state (maybe also support adding or subtracting non-lands?) and show the probability of making land drops with / without a first draw

In [12]:
def foo(num_drawable, num_lands, max_num_turns=7, is_multiplayer=False):
    fig, ax = plt.subplots(1, figsize=(12, 8))
    
    turns = np.arange(1, max_num_turns + 1)
    for num_to_draw in range(0, max_num_turns):
        ax.plot(turns,
                np.array([hypergeom.sf(num_to_draw,
                                       M=num_drawable,
                                       n=num_lands,
                                       N=7 + turn + is_multiplayer)
                          for turn in turns]),
                label='p(k >= {})'.format(num_to_draw + 1),
                ls='--', marker='o')
    
    # this is one last "hacky" curve, which is the probability
    # that we have at least 1 by turn 1, at least 2 by turn 2
    # etc
    ax.plot(turns,
            np.array([hypergeom.sf(num_to_draw,
                                   M=num_drawable,
                                   n=num_lands,
                                   N=7 + turn + is_multiplayer)
                      for (num_to_draw, turn) in enumerate(turns)]),
            label='p(k >= {})'.format(num_to_draw + 1),
            ls='--', marker='o')
        
    ax.hlines(0.5, 1, max_num_turns, ls=':', label='50%', color='black')
    ax.legend()
    ax.set_xlabel('turn')
    ax.set_ylabel('probability')

# foo(num_drawable, num_lands)

In [13]:
@interact(delta_lands=IntSlider(0, -5, 5))
def land_foo(delta_lands=0, max_num_turns=7, is_multiplayer=False):
    foo(num_drawable=num_drawable + delta_lands - 1,
        num_lands=num_lands + delta_lands,
        max_num_turns=max_num_turns,
        is_multiplayer=is_multiplayer)

interactive(children=(IntSlider(value=0, description='delta_lands', max=5, min=-5), IntSlider(value=7, descrip…